In [ ]:
%pip install duckdb

In [1]:
import duckdb

In [2]:
cn = duckdb.connect()
cn.sql("create or replace view v_detail_workouts as select * from read_csv_auto('~/concept2/workouts/*detail*.csv')")
cn.sql("create or replace view v_summary_workouts as select * from read_csv_auto('~/concept2/workouts/*summary*.csv')")

In [3]:
cn.sql("from v_detail_workouts limit 3").show()

sql = """
    select machine_type, count(*) as stroke_cnt
    , round(avg("stroke rate"),2) as avg_stroke_rate
    from v_detail_workouts
    group by 1
"""
cn.sql(sql).show()


┌────────┬────────────────┬───────────────────┬────────────────┬───────┬────────┬─────────────┬────────────┬────────┬────────────┬──────────────┐
│ Number │ Time (seconds) │ Distance (meters) │ Pace (seconds) │ Watts │ Cal/Hr │ Stroke Rate │ Heart Rate │ season │    date    │ machine_type │
│ int64  │     double     │      double       │     double     │ int64 │ int64  │    int64    │   int64    │ int64  │    date    │   varchar    │
├────────┼────────────────┼───────────────────┼────────────────┼───────┼────────┼─────────────┼────────────┼────────┼────────────┼──────────────┤
│      1 │            0.7 │               2.1 │            0.0 │  NULL │   NULL │        NULL │       NULL │   2024 │ 2024-04-03 │ rowerg       │
│      2 │            2.7 │               8.8 │          169.5 │    72 │    547 │        NULL │       NULL │   2024 │ 2024-04-03 │ rowerg       │
│      3 │            5.2 │              17.1 │          150.1 │   103 │    656 │          27 │       NULL │   2024 │ 2024-0

In [6]:
sql = """
    select 
        case 
            when grouping(year) = 1 and grouping(machine_type) = 1 then 'Grand Total'
            when grouping(year) = 0 and grouping(machine_type) = 1 then 'Sub Total'
            else cast(year as varchar)
        end as season_year,
        case 
            when grouping(machine_type) = 1 then 'All Machines'
            else machine_type
        end as machine_type,
        sum(meters) as total_meters,
        round(avg(drag_factor),2) as avg_drag_factor,
        round(avg(stroke_rate),2) as avg_stroke_rate
    from v_summary_workouts
    group by rollup(year, machine_type)
"""
cn.sql(sql).show()

cn.sql("from v_summary_workouts limit 5").show()

┌─────────────┬──────────────┬──────────────┬─────────────────┬─────────────────┐
│ season_year │ machine_type │ total_meters │ avg_drag_factor │ avg_stroke_rate │
│   varchar   │   varchar    │    int128    │     double      │     double      │
├─────────────┼──────────────┼──────────────┼─────────────────┼─────────────────┤
│ Sub Total   │ All Machines │       177200 │          106.91 │           32.74 │
│ Grand Total │ All Machines │      1143452 │          121.41 │           28.34 │
│ 2026        │ rowerg       │       146100 │           130.0 │           25.92 │
│ Sub Total   │ All Machines │       880255 │          126.14 │           27.23 │
│ Sub Total   │ All Machines │        85997 │          124.13 │            25.0 │
│ 2025        │ skierg       │        15522 │            80.4 │            41.6 │
│ 2024        │ rowerg       │        85997 │          124.13 │            25.0 │
│ 2026        │ skierg       │        31100 │            76.9 │            41.6 │
│ 2025        │ 